In [32]:
import urllib.request as ur # import para obtener la página web 
import re                   # import para utilizar expresiones regulares 

# Obtener página web con incidencias de Madrid 
f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=13&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')
s = f.read().decode()
f.close()


In [33]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [29]:
from pprint import pprint     # import para hacer prints boninitos 

dividedByTr = s.split("<tr")
endTimeElements = [] # Almacenará elementos que tienen hora fin 

# Obtener las incidencias que tienen hora fin 
print("Incidencias con hora fin")
endTime = re.compile(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",re.DOTALL)
for x in dividedByTr:
    confin = endTime.findall(x)
    #confin = re.findall(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",x,re.DOTALL)
    if len(confin): # Si el elemento contiene algún dato se agrega a la lista de elementos con hora fin
        endTimeElements.append(confin[0]) 
        pprint(confin)

# Obtener todas las incidencias 
print("\nTodas las incidencias")
conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d)?.*?(\d\d/\d\d/\d\d\d\d)?.*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(conjunto)



Incidencias con hora fin
[('06:00',
  '05/03/2019',
  '23:00',
  '05/03/2019',
  'M-111',
  'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por <b>CIRCULACIÓN</b> en<b> '
  'Túnel Barajas</b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  -  La carretera  <b> <span style="color:#ab3000">M-111 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 1.1</b> '
  'al <b> km 5</b> sentido <b>AMBOS SENTIDOS')]

Todas las incidencias
[('17:06',
  '05/03/2019',
  '',
  '',
  'A-3',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCULACIÓN</b> con circulación '
  'irregular  en<b></b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  - La AUTOPISTA / AUTOVÍA <b> <span style="color:#ab3000">A-3 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 9</b> al '
  '<b> km 8</b> sentido <b>CRECIENTE DE LA KILOMETRACIÓN</b> hacia '
  '<b>VALENCIA'),
 ('16:55',
  '05/03/2019',
  '',
  '',
  'A-1',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCUL

In [30]:
diccionario = {} # variable que se utilizará para almacenar las incidencias una sola vez 

for x in range(len(endTimeElements)):
    identificador = endTimeElements[x][0] + endTimeElements[x][1] + endTimeElements[x][4]  # concatenación de valores para crear identificador para diccionario
    diccionario.update({identificador : endTimeElements[x]}) # agregación de valores al diccionario

for x in range(len(conjunto)):
    identificador = conjunto[x][0] + conjunto[x][1] + conjunto[x][4] # concatenación de valores para crear identificador para diccionario
    if diccionario.get(identificador) == None:    # En caso de que el identificador exista, salta este paso
        diccionario.update({identificador : conjunto[x]}) # Si el identificador no existe, se crea un nuevo elemento en el diccionario
'''
Para todas las carreteras
'''
        
for x in diccionario:
    valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
    fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
    cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
    if valores[2] != '':  # En caso de que exista una hora fin
        mensaje = "Hubo un atasco en la carretera " + valores[4]
        fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
    else: 
        mensaje = "Existe un atasco en la carretera " + valores[4]
    mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
    mensaje = mensaje + fin
    mensaje = mensaje + " con la siguientes características:\n" +  cleantext
    
    print(mensaje+"\n")
    


Hubo un atasco en la carretera M-111 que inicio a las 06:00 el día 05/03/2019 el cual terminó a las 23:00 el día 05/03/2019 con la siguientes características:
RESTRICCIONES A MERCANCÍAS PELIGROSAS por CIRCULACIÓN en Túnel Barajas:    -  La carretera   M-111   a la altura de MADRID (MADRID) desde el  km 1.1 al  km 5 sentido AMBOS SENTIDOS

Existe un atasco en la carretera A-3 que inicio a las 17:06 el día 05/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  A-3   a la altura de MADRID (MADRID) desde el  km 9 al  km 8 sentido CRECIENTE DE LA KILOMETRACIÓN hacia VALENCIA

Existe un atasco en la carretera A-1 que inicio a las 16:55 el día 05/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  A-1   a la altura de MADRID (MADRID) desde el  km 9 al  km 12 sentido DECRECIENTE DE LA KILOMETRACIÓN hacia MADRID

Exis

In [31]:
'''
Para la carretera M-40 
'''

for x in diccionario:
    if "M-40" in x: 
        valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
        fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
        cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
        if valores[2] != '':  # En caso de que exista una hora fin
            mensaje = "Hubo un atasco en la carretera " + valores[4]
            fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
        else: 
            mensaje = "Existe un atasco en la carretera " + valores[4]
        mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
        mensaje = mensaje + fin
        mensaje = mensaje + " con la siguientes características:\n" +  cleantext

        print(mensaje+"\n")
    


Existe un atasco en la carretera M-40 que inicio a las 16:30 el día 05/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 12.5 al  km 10 sentido CRECIENTE DE LA KILOMETRACIÓN hacia A-3

